In [1]:
!pip install --upgrade opencv-python opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/81/3c/bbb3ceee9fbefc505f98c24dafda68c7b3c4f83b6951c0712b4623fe4cce/opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 MB 1.9 MB/s eta 0:00:24
   ---------------------------------------- 0.1/44.8 MB 1.3 MB/s eta 0:00:35
   ---------------------------------------- 0.2/44.8 MB 1.7 MB/s eta 0:00:26
   ---------------------------------------- 0.4/44.8 MB 2.6 MB/s eta 0:00:17
    --------------------------------------- 0.7/44.8 MB 3.2 MB/s eta 0:00:14
    --------------------------------------- 0.9/44.8 MB 3.7 MB/s eta 0:00:12
   - -------------------------------------- 1.2/44.8 MB 4.0 MB/s eta 0:00:11
   - -------------------------------------- 1.4/44.8 MB 4.3 MB/s eta 0:00:10
   -


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2

camera = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
kcf_tracker = None

while (1):
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    
    kcf_tracker_box = None  # результат работы трекера
    
    if kcf_tracker is not None:
        # обновляем трекер и получаем результат трекинга
        ok, box = kcf_tracker.update(frame)
        # сохраняем результат трекинга
        if ok:
            kcf_tracker_box = box
    
    # преобразуем изображение в чернобелый формат
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # запускаем детектор лиц
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)
    
    # инициализируем трекер первой детекцией
    if len(faces) != 0 and kcf_tracker is None:
        kcf_tracker = cv2.TrackerKCF_create()
        (x, y, w, h) = faces[0]
        kcf_tracker.init(frame, (x, y, w, h))
    
    for (x, y, w, h) in faces:
        # отрисовываем детекцию лиц
        cv2.rectangle(frame, (x, y), (x+w, y+h),(255, 0, 0), 2)
        cv2.putText(frame, 'Detector', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 0, 0), 2, cv2.LINE_AA)

    # отрисовываем результат трекера
    if kcf_tracker_box is not None: 
        (x, y, w, h) = map(int, kcf_tracker_box)
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Tracker', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Tracking example', frame)
    interrupt = cv2.waitKey(10)
    
    # выход по нажатию на клавишу 'q'
    if interrupt & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
camera.release()